# TC and REG Data Cleaning

## Setup

In [ ]:
# authenticate 
from google.colab import auth
auth.authenticate_user()

# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# Connect to google sheets
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Cleaning Reg

In [ ]:
import pandas as pd
import numpy as np
import csv
import re

# Date and Time
import datetime
from datetime import timedelta

# Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
import warnings
warnings.filterwarnings("ignore")

# graphs
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Import data
path = "/content/drive/My Drive/"
df_reg_raw = pd.read_csv(path + "Finder Minder/Data/new/JobReady.registrations.csv")

# replace with Google Sheet needed, already Variable summary
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1v1hhxC23KJ7t-S26MAwBpCqo1JR2mdrvzuzMWt7_hPU/edit#gid=897172359'
wb = gc.open_by_url(google_sheet_url)

# get Google Sheet worksheet
sheet1 = wb.worksheet('JobReady Reg')
data = sheet1.get_all_values()
regVarSummary = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
# helper functions

def StringToDate(df, cols, date_format='%Y-%m-%d %H:%M:%S', to_float=False):
  new_df = pd.DataFrame(columns = cols)
  for col in cols:
    try:
      new_df[col] = pd.to_datetime(df[col], format = date_format, errors='coerce')
      if to_float == True:
        new_df[col] = new_df[col].apply(lambda date: (date - datetime.datetime(1970, 1, 1)) / datetime.timedelta(days=1))#.astype('float')
    except:
      print(col)
  return new_df
  

def ColumnsWithTreatmentReg(treatment):
    return list(regVarSummary['ColName'][regVarSummary['Treatment'] == treatment])


# given df and columns to check, get missing values percentage into a dictionary
def MissingCounts(df, cols): 
    missingPercent = {}
    for col in cols:
        missingPercent[col] = round(df[col].isnull().sum() / len(df), 2)
    return missingPercent

# prints value counts and percent missing in sorted missing
def PrintStats(df, missingDict):

    for key, value in sorted(missingDict.items(), key=lambda item: item[1]):
        print(key)
        print("-"*len(key))
        
        print(df[key].value_counts(dropna = False))
        print(df[key].value_counts(dropna = False, normalize = True) * 100)
        print('missing %f \n' % value)

# creates columns for date
def SeparateDate(df, columns):
    for col in columns:
        df[col + 'year'] = pd.DatetimeIndex(df[col]).year
        df[col + 'month'] = pd.DatetimeIndex(df[col]).month
        df[col + 'day'] = pd.DatetimeIndex(df[col]).day
        df.drop(col, axis=1, inplace=True)
    return df

In [ ]:
#################
# Summary
#################

# label encoder can turn values in columns into column_value columns
# put all cols to drop in this array
dropCols = []

#################
# Drop cols
#################

df_reg = df_reg_raw.dropna(1,thresh = int(len(df_reg_raw)*0.01)) # 99% missing
dropVariableSummary =  ColumnsWithTreatment('DROP')
dropIdenticalValues = [col for col in df_reg.columns if (df_reg[col].nunique() == 1) & (df_reg[col].isnull().sum() == 0)]
dropOther = ["LoadDateTime",              # SQL query data 
             "notinworkforcefor3years"]   # Indicator w/out info (all 0 or missing)
df_reg.drop(dropVariableSummary + dropIdenticalValues + dropOther, axis=1, inplace=True)

# Drop rows without target variable, still ongoing apprenticeships
df_reg = df_reg[df_reg['actualenddate'].isnull().eq(False)]

# get target variable and econde
df_reg['GRADUATED'] = np.where(df_reg['endreason'] == 'CMPS', 1, 0)

#################
# Read datetimes
#################

# drop greater than 90 missing

dateCols = ColumnsWithTreatment('DATE') # get cols with date treatment
df_reg[dateCols] = StringToDate(df_reg, dateCols,'%Y-%m-%d') # convert them all to datetime format

# replace values of expectedenddate with expected enddate overrride (updated expected enddate), ignore 
df_reg['expectedenddate'] = np.where(df_reg['expectedenddate_override'].isnull().eq(False), df_reg['expectedenddate_override'], df_reg['expectedenddate'])

# get number of years to complete
df_reg['YEARS_TO_COMPLETE'] = (df_reg['actualenddate'] - df_reg['startdate']).astype('timedelta64[D]')/365

# removes bad years but also some 0's are actually 1s so need a better way to find years to complete
df_reg = df_reg[df_reg['YEARS_TO_COMPLETE'] >= 0].reset_index(drop=True)

# actualenddate - 0.0000
# commencementvisitdate - 0.230000, commencementvisitdate around the same as start date, but not duplicate
# recommencementdate - 0.830000  
# sixmonthspointdate - 0.300000, on average it is actually 6 months
# twelvemonthpointdate - 0.670000 on average it is actually 12 months
# twentyfourmonthpointdate - 0.840000 on average it is actually 24 months
# initialservicingdateofcontact - 0.420000 
# intermediateservicingdateofcontact - 0.71000 
# workplacefromdate - 0.320000 Experience at employer
# sta_approved_date - 0.420000 
# sta_registered_date - 0.430000 Check if these events occur before completion/dropout, on average it seems that sta_registered_date is before actualenddate
# 2207-11-03 should be 2007-11-03, change 1900-01-01 to null

df_reg['sta_registered_date'] = df_reg['sta_registered_date'].mask(df_reg['sta_registered_date'].dt.year == 1900, 
                                                                   np.datetime64('NaT'))

# daterecievedfromsta - 0.510000 Check if these events occur before completion/dropout, on average it seems that daterecievedfromsta is after actualenddate
# sta_notified_end_date - 0.660000 on average it seems that sta_notified_end_date is about same as expectedenddate, 2107-01-13 should be 2017-01-13
df_reg['daterecievedfromsta'] = df_reg['daterecievedfromsta'].mask(df_reg['daterecievedfromsta'].dt.year == 2107, 
                                                                   df_reg['daterecievedfromsta'] + pd.offsets.DateOffset(year=2017))

df_reg['daterecievedfromsta'] = df_reg['daterecievedfromsta'].mask(df_reg['daterecievedfromsta'].dt.year == 2208, 
                                                                   df_reg['daterecievedfromsta'] + pd.offsets.DateOffset(year=2008))
df_reg[dateCols] = StringToDate(df_reg, dateCols,'%Y-%m-%d')
dropCols += ['expectedenddate_override']

#################
# Binary
#################

# checking binary
binaryCols = ColumnsWithTreatment('BINARY')

# filling in all the missing values to -99
# for col in binaryCols:
#     df[col] = df[col].fillna(value = -99)

# drop by default those over 90%
dropCols += ['isthisaustralianapprenticeshipsupportedbytheschool', 'assessedasawtuineligible', 
             'ineligible_supportforadultaustralianapprentices', 'saaa_tfn_supplied', 'redundant', 
             'intensivesupportassistance', 'incomesupport', 'twentyfourmonthmodeofcontact']

#################
# Small Cat 
#################

# Strip and lower all string fields
df_obj = df_reg.select_dtypes(['object']).apply(lambda x: x.str.lower())
df_reg[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

# Check small category cleaning
colsSmallCat = ColumnsWithTreatment('SMALL_CAT')
missingSCat = MissingCounts(df_reg, colsSmallCat)

# standardising
# getting rid of prefix
df_reg['aqflevel'] = df_reg['aqflevel'].str.strip('aqf')

# fixing formatting for sta
df_reg['sta'] = df_reg['sta'].str.replace("-sta", "")
df_reg['sta'] = df_reg['sta'].str.strip()

# shortening stacommunicationstatusmessage for encoding
df_reg['stacommunicationstatusmessage'] = df_reg['stacommunicationstatusmessage'].map({'details amended or entered by sta': 3,
                                                                                       'sta confirmation recieved': 2,
                                                                                       'details amended by sta during confirmation': 4,
                                                                                       'sta confirmation pending': 1}) 

# filling in nas with -99
# for col in colsSmallCat:
#     df_reg[col] = df_reg[col].fillna(value = -99)

# dropping over 90% missing
dropCols += ['saaa_recip', 'assessedasctlsineligiblecomments', 'assessedasawtuineligiblecomments']


#################
# Nums
#################

# Check large category cleaning
nums = ColumnsWithTreatment('NUMERICAL')

# nominaldurationinmonths 0.32 nominal duration for what?

# creating categories for hoursperweek 0.87, according to maximum hours for apprenticeship Google search max per week is 38 hours
df_reg['overmaxhoursperweek'] = np.where(df_reg['hoursperweek'] > 38, 1, 0)
# high hours
df_reg['highhoursperweek'] = np.where((df_reg['hoursperweek'] <= 38) & (df_reg['hoursperweek'] > 16), 1, 0)
# two days is 16 hours
df_reg['mediumhoursperweek'] = np.where((df_reg['hoursperweek'] <= 16) & (df_reg['hoursperweek'] > 8), 1, 0)
# normal day is 8 hours, 9-5
df_reg['lowhoursperweek'] = np.where(df_reg['hoursperweek'] <= 8, 1, 0)

# numberofemployeesinworkplace 0.32 - group into large, medium small companies
# larger than 75th quantile is big
# 25th to 75th quantile is medium
# under 25th quantile small
df_reg['at_small_company'] = np.where(df_reg['hoursperweek'] <= 10, 1, 0)
df_reg['at_medium_company'] = np.where((df_reg['hoursperweek'] <= 450) & (df_reg['hoursperweek'] > 10), 1, 0)
df_reg['at_big_company'] = np.where(df_reg['hoursperweek'] > 450, 1, 0)

# ftequivalentmonths  0.87 full time equivalent months
df_reg['ft_small'] = np.where(df_reg['ftequivalentmonths'] <= 9, 1, 0)
df_reg['ft_medium'] = np.where((df_reg['ftequivalentmonths'] <= 61) & (df_reg['hoursperweek'] > 9), 1, 0)
df_reg['ft_big'] = np.where(df_reg['ftequivalentmonths'] > 61, 1, 0)

# filling in nas with -99
#for col in colsLargeCat:
#    df_reg[col] = df_reg[col].fillna(value = -99)

# no variables over 90% missing

#################
# Large Cat 
#################

# Check large category cleaning
colsLargeCat = ColumnsWithTreatment('LARGE_CAT')

# employer_id 0.0 seems to have different lengths, but thats ok, unique identifier and could be used for econ data merge
# endreason 0.0 keep as it is important

# split qualification code into prefix and suffix
df_reg['qc_prefix'] = df_reg['qualificationcode'].map(lambda x: x[:2])
df_reg['qc_suffix'] = df_reg['qualificationcode'].map(lambda x: x[2:])

# dropping tyims_created_by, tyims_updated_by as they are created by the system, and are system operations, not necessary
dropCols += ['tyims_created_by', 'tyims_updated_by']

# dropping workplaceaddline1 as there are many variables there that are uncategorisable and location information is already in workplacepostcode
dropCols += ['workplaceaddline1']
# try to standardise and group workplaceemployername
# map all employer ids to one workplaceemployername, deduplicate
match = {}
for i in df_reg['employer_id'].unique():
    for idx, val in df_reg[df_reg['employer_id'] == i]['workplaceemployername'].value_counts().iteritems():
        match[i] = idx
        break

df_reg['workplaceemployername'] = df_reg['employer_id'].map(match).fillna(df_reg['workplaceemployername'])

# workplacepostcode 0.0 information about location, can replace workplacesuburb and is good enough for workplaceaddline1
# dropping workplacesuburb as this is basically same as workplacepostcode
dropCols += ['workplacesuburb']

# staid - 0.01 split to prefix and suffix, e.g. 2269481\3
df_reg['staid_prefix'], df_reg['staid_suffix'] = df_reg['staid'].str.split('\\',2).str

# rto_id 0.01 seems to have different lengths for id, not sure why RTO Registered Training Organisation

# barcode 0.02 not sure what for, aac12/98268, 372489
df_reg['barcode_prefix'] = df_reg['barcode'].astype(str).str[:5]
df_reg['barcode_suffix'] = df_reg['barcode'].astype(str).str[6:]

# deewrfilenumber 0.02 Department of Education, Employment and Workplace Relations, aac10/319930, 372489
df_reg['deewrfilenumber_prefix'] = df_reg['deewrfilenumber'].astype(str).str[:5]
df_reg['deewrfilenumber_suffix'] = df_reg['deewrfilenumber'].astype(str).str[6:]

# commencementvisitfieldofficer 0.24 diago rodrigues, for each commencement visit field officer, create id, create exist binary
df_reg['ITC_experience'] = np.where(df_reg['commencementvisitfieldofficer'].isnull(), 0, 1)


# employername 0.32 repeated and weaker version of workplaceemployername, more missing, same info, drop
dropCols += ['employername']

# tyims_contractid 0.32, keep as it could be useful for merging trc, mapping contract details

# anzscooccupationcode 0.32, keep and do nothing, might be useful no formatting needed

# site_code 0.35, 64, keep might be useful

# apprenticeshipqualificationid 0.36, 3525 might be useful keep

# registration_manager_id 0.36, 51, might be useful keep as an id

# employerbankaccountidlinkedtoreg 0.61, 41780, employerhasbankaccount binary relates
# but double missing so drop, no need for bank accounts details just employer details that don't describe the employer, drop
dropCols += ['employerbankaccountidlinkedtoreg']

# clientcontactphone 0.79, 73137, a lot of unique values, no relevance to apprenticeship, just employer details that don't describe the employer
# area code can be found with postcode
dropCols += ['clientcontactphone']

# sta_notified_end_date_updated_by 0.87, 30, weird values but keep

# certificatenumber 0.88, 45190, many unique numbers, as well as missing values and this probably happens after Traineeship/apprenticeship
# dropCols += ['certificatenumber']
# feature engineer says keep for some reason so i guess ill keep

# recommencementvisitfieldofficer 0.93, 1420, exception will keep, somehow separate recommenced apprentices

# filling in nas with -99
# for col in colsLargeCat:
#     df_reg[col] = df_reg[col].fillna(value = -99)

# dropping more than 90% missing - ineligible_employer_reason_type_id 0.95
dropCols += ['ineligible_employer_reason_type_id']

# drop columns and merge binary to the df_reg
df_reg.drop(columns = dropCols, axis=1, inplace=True)
df_reg.drop(columns = ['employee_id'], axis=1, inplace=True)

# export the final dataset
df_reg.to_csv('registrations_cleaned_v2.csv', index=False)

!cp registrations_cleaned_v2.csv "/content/drive/My Drive/Finder Minder/Data/new/"

# Cleaning TRC

In [ ]:
# import variable summary
file_url = "https://docs.google.com/spreadsheets/d/1v1hhxC23KJ7t-S26MAwBpCqo1JR2mdrvzuzMWt7_hPU/edit#gid=1634553018"
wb = gc.open_by_url(file_url)
sheet_name = 'JobReady TCs'
variableSummary = pd.DataFrame(wb.worksheet(sheet_name).get_all_values()[1:],
                               columns = wb.worksheet(sheet_name).get_all_values()[0])

# Import data
path = "/content/drive/My Drive/Finder Minder/"
df_trc_raw = pd.read_csv(path + "Data/new/JobReady.training_contracts.csv")
df_trc = pd.read_csv("/content/drive/My Drive/Finder Minder/Data/new/training_contract_partial_clean.csv")

In [ ]:
def StringToDate(df, cols, date_format='%Y-%m-%d %H:%M:%S', to_float=False):
  new_df = pd.DataFrame(columns = cols)
  for col in cols:
    try:
      new_df[col] = pd.to_datetime(df[col], format = date_format, errors='coerce')
      if to_float == True:
        new_df[col] = new_df[col].apply(lambda date: (date - datetime.datetime(1970, 1, 1)) / datetime.timedelta(days=1))#.astype('float')
    except:
      print(col)
  return new_df

def ColumnsWithTreatment(treatment):
    return list(variableSummary['ColName'][variableSummary['Treatment'] == treatment])

def PrintValueCounts(df, cols):
    for col in cols:
        print(col)
        print("-"*len(col))
        print(df[col].value_counts())
        print("missing", df[col].isnull().mean())
        print()

def EncodeIfStringContains(df, col, substring):
    """
    Input:
        df = (dataframe)
        col = (string) column name
        substring = (list[string]) a list of the substrings you want to check if it contains
    """
    for sub in substring:
        df[col+"_"+substring] = np.where(df[col].str.contains(sub), 1, 0)
print(df_trc.shape)

In [ ]:
#################
# Drop cols
#################

df_trc = df_trc_raw.dropna(1,thresh = int(len(df_trc_raw)*0.01)) # 99% missing

# Converting Missing column to float
variableSummary['Missing'] = pd.to_numeric(variableSummary['Missing'])

dropNullCols= list(variableSummary['ColName'][variableSummary['Missing'] >= 0.90])

dropVariableSummary =  ColumnsWithTreatment('DROP')
#print(dropVariableSummary)
dropIdenticalValues = [col for col in df_trc.columns if (df_trc[col].nunique() == 1) & (df_trc[col].isnull().sum() == 0)]
#dropOther = ["LoadDateTime", "notinworkforcefor3years"]   # Indicator w/out info (all 0 or missing)
#print(dropIdenticalValues)
df_trc.drop(dropVariableSummary + dropIdenticalValues, axis=1, inplace=True)

In [ ]:
# binary
binaryCols = ColumnsWithTreatment('BINARY')
for i in binaryCols:
    if i in df_trc.columns:
        df_trc[i].fillna(-99, inplace=True)

In [ ]:
date_cols = ColumnsWithTreatment('DATE')
df_trc[date_cols] = StringToDate(df_trc, date_cols,'%Y-%m-%d')


# Ensuring all the data types have been converted
for i in date_cols:
    if i in df_trc.columns:
        print(df_trc[i].dtype)

In [ ]:
small_cat_cols = ColumnsWithTreatment('SMALL_CAT')

In [ ]:
df_trc.guardianpostalstate = df_trc_raw.guardianpostalstate

state_dict = {'NSW': 0, 'QLD': 1, 'ACT': 2, 'VIC': 3, 'WA': 4, 'TAS': 5, 'SA': 5, 'NT': 6}

for i in df_trc.employerpostalstate.astype('str').unique():
    for j in state_dict.keys():
        if re.search(j, i, re.IGNORECASE):
            #df_trc.at[i,'employerpostalstate']= j
            df_trc.loc[df_trc['employerpostalstate'] == i, 'employerpostalstate'] = j
df_trc.employerpostalstate.fillna(-99)
df_trc.loc[df_trc['employerpostalstate'] == " ", 'employerpostalstate'] = -99

In [ ]:
df_trc.typeofemploymentarrangement = df_trc_raw.typeofemploymentarrangement

for idx, val in df_trc['typeofemploymentarrangement'].value_counts().iteritems():
    if val < 4000:
        df_trc['typeofemploymentarrangement'][df_trc['typeofemploymentarrangement'] == idx] = -99

df_trc.typeofemploymentarrangement.fillna(-99)

In [ ]:
# Strip and lower all string fields
df_obj = df_trc.select_dtypes(['object']).apply(lambda x: x.str.lower())
df_trc[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
drop = []

# Apprenticeship type
# Encoding Traineeship as 0 and Apprenticeship as 1
df_trc['typeofapprenticeship'] = df_trc['typeofapprenticeship'].map({'T': 0, 'A': 1, 'Traineeship': 0, 'Apprenticeship': 1, 'S': 2, 'Trainee Apprentice (NSW Only)': 3})
# Replacing missing values with -99
df_trc['typeofapprenticeship'].fillna(-99)

# Employer type
# df_trc['employertype'] = df_trc['employertype'].astype(str)
# Filling null values with -99
df_trc['employertype'].fillna(-99)

# Attendence type: Full-Time is 1, Part-Time is 0, School is 2, missing values -99
df_trc['attendancetype'] = df_trc['attendancetype'].map({'Full-Time': 1, 'Part-Time': 0, '1.0': 1, '2.0': 0, 'S': 2, 'School-Based': 2})
df_trc['attendancetype'].fillna(-99)

# Current year
df_trc['attendingschoollevel_senior'] = np.where(df_trc['attendingschoollevel'].isin([11,12]), 1, 0)
df_trc['attendingschoollevel_junior'] = np.where(df_trc['attendingschoollevel'] <= 10, 1, 0)
drop += ['highestcompletedschoollevel']

# Highest completed year
df_trc['highestcompletedschoollevel_graduated'] = np.where(df_trc['highestcompletedschoollevel'] == 12, 1, 0)
df_trc['highestcompletedschoollevel_senior'] = np.where(df_trc['highestcompletedschoollevel'].isin([10,11]), 1, 0)
df_trc['highestcompletedschoollevel_junior'] = np.where(df_trc['highestcompletedschoollevel'] <= 9, 1, 0)
df_trc['highestcompletedschoollevel_graduated'].fillna(-99)
df_trc['highestcompletedschoollevel_senior'].fillna(-99)
df_trc['highestcompletedschoollevel_junior'].fillna(-99)
drop += ['highestcompletedschoollevel']

# States

state_dict = {'NSW': 0, 'QLD': 1, 'ACT': 2, 'VIC': 3, 'WA': 4, 'TAS': 5, 'SA': 5, 'NT': 6}
df_trc['guardianpostalstate'] = df_trc['guardianpostalstate'].map(state_dict)
df_trc['guardianpostalstate'].fillna(-99)

for i in df_trc.employerpostalstate.astype('str').unique():
    for j in state_dict.keys():
        if re.search(j, i, re.IGNORECASE):
            df_trc.loc[df_trc['employerpostalstate'] == i, 'employerpostalstate'] = j
df_trc.employerpostalstate.fillna(-99)
df_trc.loc[df_trc['employerpostalstate'] == " ", 'employerpostalstate'] = -99

for i in df_trc.appresidentialstate.astype('str').unique():
    for j in state_dict.keys():
        if re.search(j, i, re.IGNORECASE):
            df_trc.loc[df_trc['appresidentialstate'] == i, 'appresidentialstate'] = j
df_trc.appresidentialstate.fillna(-99)
df_trc.loc[df_trc['appresidentialstate'] == " ", 'appresidentialstate'] = -99

for i in df_trc.workplacestate.astype('str').unique():
    for j in state_dict.keys():
        if re.search(j, i, re.IGNORECASE):
            df_trc.loc[df_trc['workplacestate'] == i, 'workplacestate'] = j
df_trc.workplacestate.fillna(-99)
df_trc.loc[df_trc['workplacestate'] == " ", 'workplacestate'] = -99

# Indigenous
df_trc['indigenousstatus'].fillna(-99)

# Remove categories with small count
# for val in df_trc['modeofdeliveryid'].unique():
#     if df_trc['modeofdeliveryid'][df_trc['modeofdeliveryid'] == val].sum() < 10000:
#         df_trc['modeofdeliveryid'][df_trc['modeofdeliveryid'] == val] = np.NaN
df_trc.modeofdeliveryid.fillna(-99)

# Sex and nationality
df_trc['appsex'] = np.where(df_trc['appsex'].isin(['M','Male']), 0, 1)
df_trc['appcitizenship'] = np.where(df_trc['appcitizenship'] == 'a', 0, 1)

# employment arrangement
for idx, val in df_trc['typeofemploymentarrangement'].value_counts().iteritems():
    if val < 4000:
        df_trc['typeofemploymentarrangement'][df_trc['typeofemploymentarrangement'] == idx] = -99
#print(df_trc.typeofemploymentarrangement.value_counts())
df_trc.typeofemploymentarrangement.fillna(-99)

df_trc.employertype.fillna(-99)

In [ ]:
# numeric cols
numericCols = ColumnsWithTreatment('NUMERICAL')
for i in numericCols:
    if i in df_trc.columns:
        # print(i)
        df_trc[i].fillna(-99, inplace=True)

In [ ]:
# large cat
largeCols = ColumnsWithTreatment('LARGE_CAT')
for i in largeCols:
    if i in df_trc.columns:
        df_trc[i].fillna(-99, inplace=True)

In [ ]:
df_trc['approvingsta'].value_counts()
#df_trc['approvingsta'].apply(str).apply(lambda x: x.str.split('-')[0])

df_trc['approvingsta'].apply(str)
for idx, val in df_trc['approvingsta'].astype('str').iteritems():
    if str(val).split('-'):
        states = str(val).split('-')[0]
        df_trc.loc[idx, 'approvingsta'] = states
        #df_trc.loc[df_trc['approvingsta'] == i, 'employerpostalstate'] = j

In [ ]:
df_trc['approvingsta'].value_counts()

for i in df_trc.approvingsta.astype('str').unique():
    for j in state_dict.keys():
        if re.search(j, i, re.IGNORECASE):
            df_trc.loc[df_trc['approvingsta'] == i, 'approvingsta'] = j
df_trc.approvingsta.fillna(-99)
df_trc['approvingsta'] = df_trc['approvingsta'].astype('str').replace({'1320-sta': -99, 'nan': -99})

In [ ]:
EncodeIfStringContains(df_trc, 'qualificationlevelobtained', ['certificate', 'ii', 'iii', 'child', 'bachelor', 'engineering', 'construction'])

In [ ]:
df_trc.to_csv('TC_clean1.csv', index=False)
!cp TC_clean1.csv "drive/My Drive/Finder Minder/Data"